In [ ]:
%cd /Users/santiago/thesis/code

# reload magic
%load_ext autoreload
%autoreload 2

from src import utils
from src import jit_dtw # jitted dtw
from src import dtw as v_dtw # vanilla dtw
from src import c_dtw   # wrapper C dtw
from src import c_path_dtw
import dtw as g_dtw # Giorgino's dtw

import time
import numpy as np

In [ ]:
datasets = utils.get_n_datasets(10)
S = utils.get_class_sequences(2, datasets[1])

c, P = jit_dtw.dtw(S[0, :50], S[1, :50])
wp = c_path_dtw.dtw_path(S[0, :50], S[1, :50])

for i in range(P.shape[0]):
    assert P[i, 0] == wp[i, 0] and P[i, 1] == wp[i, 1]

t0 = utils.time_execution(jit_dtw.dtw, S[0], S[1])
t1 = utils.time_execution(c_path_dtw.dtw_path, S[0], S[1])
t2 = utils.time_execution(v_dtw.dtw, S[0], S[1])
t3 = utils.time_execution(g_dtw.dtw, S[0], S[1])

print(t0)
print(t1)
print(t2)
print(t3)

# t0 = utils.time_execution(jit_dtw.dtw_cost, S[0], S[1])
# t1 = utils.time_execution(c_dtw.dtw_cost, S[0], S[1])


# print(t0)
# print(t1)
# print(t2)
# print(t3)

In [ ]:
'''Execution time analysis on three DTW methods:
    * my vanilla DTW
    * my jit DTW
    * Giorgino's DTW
'''

datasets = utils.get_n_datasets(10)

def time_execution(foo, a, b):
    start = time.time()
    foo(a, b)
    end = time.time()
    interval = round((end - start)*10**3, 2)
    return interval

for dataset in datasets:
    n = 10
    total_t1 = 0
    total_t2 = 0
    total_t3 = 0

    S = utils.get_class_sequences(n+1, dataset) # get 11 sequences from same (random) class
    n = S.shape[0]

    # Do DTW 10 times
    for i in range(1, n):
        # total_t1 += time_execution(g_dtw.dtw, S[0], S[i])
        total_t1 += time_execution(dtw.dtw, S[0], S[i])
        total_t2 += time_execution(jit_dtw.dtw, S[0], S[i])
        total_t3 += time_execution(g_dtw.dtw, S[0], S[i])
    
    t1_avg = round(total_t1/(n-1), 2)
    t2_avg = round(total_t2/(n-1), 2)
    t3_avg = round(total_t3/(n-1), 2)
    print(f'Seq length: {S.shape[1]}\t{t1_avg}ms\t{t2_avg}ms\t{t3_avg}ms')